<a href="https://colab.research.google.com/github/wetdog/insanely-fast-whisper/blob/main/4bit_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#4bit quantization benchmark

Additional to the optimizations made using batching and bettertransformer, we can do quantization optimizations. We would test this strategies with the same audio that was used on the *transformers* 🤗 and fast-whisper benchmarks.


In [ ]:
!wget https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/sam_altman_lex_podcast_367.flac
!ffmpeg -y -i sam_altman_lex_podcast_367.flac -ac 1 -ar 16000 sam_altman_lex_podcast_367.wav



--2023-10-24 22:10:15--  https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/sam_altman_lex_podcast_367.flac
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/96/e4/96e4f69cd112b019dd764318570e47e5fe96de53d8c32a99d745e72d9086e355/b2fd593ce144a8d904cf49a4ed77ed06eb50644a053dddd280c81a3ef94fb60e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sam_altman_lex_podcast_367.flac%3B+filename%3D%22sam_altman_lex_podcast_367.flac%22%3B&response-content-type=audio%2Fx-flac&Expires=1698444615&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5ODQ0NDYxNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85Ni9lNC85NmU0ZjY5Y2QxMTJiMDE5ZGQ3NjQzMTg1NzBlNDdlNWZlOTZkZTUzZDhjMzJhOTlkNzQ1ZTcyZDkwODZl

First install al the requirements

In [ ]:
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q -U git+https://github.com/huggingface/transformers
!pip install -q -U accelerate optimum
!pip install -q ipython-autotime
!pip install -q -U bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 19.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00


let's load the 3 model whisper parts: Tokenizer, Feature extractor and the model itself with the 4bit quantization option.

We can then pass each of this elements as arguments to the automatic speech recognition pipeline

In [ ]:
from transformers import (WhisperForConditionalGeneration,
                         WhisperFeatureExtractor,
                         WhisperTokenizer,
                        pipeline,
                        BitsAndBytesConfig)
from transformers.configuration_utils import Dict
import torch
import soundfile as sf

#load feature extractor
extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")
# load tokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="en", task="transcribe")
#load model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2",
                                                        load_in_4bit=True,
                                                        bnb_4bit_compute_dtype=torch.float16)


pipe = pipeline("automatic-speech-recognition",
                model=model,
                feature_extractor=extractor,
                tokenizer=tokenizer,
                torch_dtype=torch.float16,
                )

# load audio
x, fs= sf.read("sam_altman_lex_podcast_367.wav")

In [ ]:
%%time


outputs = pipe(inputs= {"raw":x,"sampling_rate": fs},
                chunk_length_s=30,
                batch_size=24,
                return_timestamps=True)

CPU times: user 8min 15s, sys: 27.2 s, total: 8min 42s
Wall time: 8min 35s


Now that the model have a smaller memory footprint, we can try to increase the batchsize.

In [ ]:
## Using larger batchsize
%%time

outputs = pipe(inputs= {"raw":x,"sampling_rate": fs},
              chunk_length_s=30,
              batch_size=40,
              return_timestamps=True)

CPU times: user 6min 22s, sys: 26.7 s, total: 6min 49s
Wall time: 6min 29s
